In [1]:
"""
Probably gonna need to update this in future.
"""

objectInfoDict={'m82'     :{'redshift'    :0.000677,
                            'centerCoo'   :(148.9667,69.6794),
                            '1arcScaleBar':'17.5 pc',
                            'inclination' : 81,
                            'majorPA'     : 65,
                            'distance'    : 3.6},
                'cenA'    :{'redshift'    :0.001825,
                            'centerCoo'   :(201.3650633,-43.0191125),
                            '1arcScaleBar':'18.6 pc',
                            'inclination' : 75,
                            'majorPA'     : 145,
                            'distance'    : 3.84},
                'circinus':{'redshift'    :0.001448,
                            'centerCoo'   :(213.29147083,-65.33921861),
                            '1arcScaleBar':'20.4 pc',
                            'inclination' : 65,
                            'majorPA'     : 216,
                            'distance'    : 4.2},
                'ngc253'  :{'redshift'    :0.000811,
                            'centerCoo'   :(11.888002,-25.288220),
                            '1arcScaleBar':'20 pc',
                            'inclination' : 76,
                            'majorPA'     : 51,
                            'distance'    : 3.9},
                'ngc1068' :{'redshift'    :0.003793,
                            'centerCoo'   :(40.6696292,-0.0132806),
                            '1arcScaleBar':'72 pc',
                            'inclination' : 40,
                            'majorPA'     : 30,
                            'distance'    : 14.4},
                'ngc3079' :{'redshift'    :0.003723,
                            'centerCoo'   :(150.4908475, 55.6797894),
                            '1arcScaleBar':'85 pc',
                            'inclination' : 84,
                            'majorPA'     : 166,
                            'distance'    : 16},
                'ngc4945' :{'redshift'    :0.001878,
                            'centerCoo'   :(196.3644897,-49.4682129),
                            '1arcScaleBar':'20 pc',
                            'inclination' : 75,
                            'majorPA'     : 45,
                            'distance'    : 3.8},
                'ngc6240' :{'redshift'    :0.024480,
                            'centerCoo'   :(253.245295,2.400926),
                            '1arcScaleBar':'722 pc',
                            'inclination' : '?',
                            'majorPA'     : '?',
                            'distance'    : 147}
               }

from dictionaryThings import saveDict
saveDict(dictionary=objectInfoDict,saveName='/Volumes/QbertPrimary/umdResearch/adapProposalNearby/objectInfoDict.pkl')

In [ ]:
"""
    This is a script which utilizes the ObsInfo class.

    Create an instance of ObsInfo using data and parameters from a single Herschel
    observation. Compute the reference velocity for the spectral axis, 
    update FITS header keywords, and save to new FITS file.
    
    Create the params.txt input file for BBarolo.
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard
import os
import numpy as np
# Related
from astropy.wcs import WCS
# Local
from ObsInfo import ObsInfo
from dictionaryThings import loadDict
from functionThings import waveToVel

In [5]:
# Spaxel size
arcsec = '1arc'

# Which program will be reading the output FITS file?
program = '3dBarolo'
#program = 'kpvslice'

topPath = '/Volumes/QbertPrimary/umdResearch/adapProposalNearby/'


# ----------------------------------------- #
# Necessary parameter file and dictionaries #
# ----------------------------------------- #
# Dictionary of galaxy properties
objDictName = topPath+'objectInfoDict.pkl'
# Dictionary of emission line properties.
lineDict = loadDict(topPath+'emiLineDict.pkl')
# Text file of line fitting parameters.
paramFileName = topPath + 'fittingParametersV4.txt'


# -------------------------------------------------- #
# Read in the parameter file containing line profile #
# velocity limits and continuum fitting information. #
# -------------------------------------------------- #
paramFileData = np.genfromtxt(paramFileName, dtype=None, autostrip=True,
                              names=True, encoding=None)

In [6]:
for x in range(len(paramFileData)):
    # ------------------------------------ #
    # Get the galaxy and line information. #
    # ------------------------------------ #
    obsInfo = ObsInfo(x, paramFileName, objDictName)
    
    # Name of the emission line.
    lineName = paramFileData['lineNameShort'][x]
    # Rest wavelength of the emission line.
    restWave = lineDict[lineName]['restWave']
    sysWave = (1.+obsInfo.z)*restWave
    # Spatial resolution at line wavelength
    spatRes = lineDict[lineName]['spatRes']


    # ----------------------------------------- #
    # Create the base names for saving outputs. #
    # ----------------------------------------- #
    # Base for the object's file names.
    objectNameBase = (str(obsInfo.obsId)+'_'+obsInfo.objectName
                      +'_'+lineName)
    
    # Base path to the object's folders.
    objectPathBase = (topPath+'pySpecKitCube/run4/'+obsInfo.objectName
                          +'/'+arcsec+'/'+program+'/')
    if (not os.path.exists(objectPathBase)):os.makedirs(objectPathBase)



    # -------------------------------------------------------------- #
    # Find the minimum velocity of the spectral axis.                #
    # This will be set as the reference velocity in the FITS header. #
    # -------------------------------------------------------------- #
    fitsFile = (topPath+'pySpecKitCube/run4/'+obsInfo.objectName+'/'
                +arcsec+'/outFitsFiles/'+objectNameBase+'.fits')

    if program == 'kpvslice':
        velCorr = True
    if program == '3dBarolo':
        velCorr = False

    obsInfo.computeVels(fitsFile, restWave, velCorr=velCorr)


    # ----------------------------------------------------- #
    # Update/add spectral axis keywords in the FITS header. #
    # Also add beamsize information.                        #
    # ----------------------------------------------------- #
    fitsSavePath = objectPathBase + 'inFits/'
    if (not os.path.exists(fitsSavePath)):os.makedirs(fitsSavePath)
    
    fitsSaveName = fitsSavePath+objectNameBase+'_hdrEditVel.fits'
    
    obsInfo.updateHdr(fitsFile, restWave, beamSize=spatRes, saveFile=fitsSaveName)


    if program == '3dBarolo':
        # --------------------------- #
        # Parameters for barolo input #
        # --------------------------- #
        raCenter, decCenter = obsInfo.raCenter, obsInfo.decCenter
        w = WCS(obsInfo.hdr).celestial
        xPos, yPos = w.wcs_world2pix(raCenter, decCenter, 1)
        vSys = int(waveToVel(sysWave, restWave))


        # ------------------------ #
        # Save BBarolo parameters. #
        # ------------------------ #
        paramFilePath = objectPathBase + lineName + '/'
        if (not os.path.exists(paramFilePath)):os.makedirs(paramFilePath)

        outFile = open(paramFilePath + 'params.txt', 'w')
        outFile.write('FITSFILE    ' + fitsSaveName + '\n'+
                      'OUTFOLDER   ' + objectPathBase + lineName + '/\n'+
                      '{0:<10s}'.format('THREADS') + '{0:<8s}'.format('4') +'\n'+
                      '{0:<10s}'.format('BEAMFWHM') + '{0:<8s}'.format(str(spatRes)) +'\n'+
                      '{0:<10s}'.format('GALFIT') + '{0:<8s}'.format('TRUE') +'\n'+
                      '{0:<10s}'.format('NRADII') + '{0:<8s}'.format('5') +'\n'+
                      '{0:<10s}'.format('RADSEP') + '{0:<8s}'.format('4') +'\n'+
                      '{0:<10s}'.format('VSYS') + '{0:<8s}'.format(str(vSys)) +'\n'+
                      '{0:<10s}'.format('XPOS') + '{0:<8s}'.format(str(int(xPos))) +'\n'+
                      '{0:<10s}'.format('YPOS') + '{0:<8s}'.format(str(int(yPos))) +'\n'+
                      '{0:<10s}'.format('VROT') + '{0:<8s}'.format('60') +'\n'+
                      '{0:<10s}'.format('VDISP') + '{0:<8s}'.format('100') +'\n'+
                      '{0:<10s}'.format('INC') + '{0:<8s}'.format(str(obsInfo.inc)) +'\n'+
                      '{0:<10s}'.format('DELTAINC') + '{0:<8s}'.format('15') +'\n'+
                      '{0:<10s}'.format('PA') + '{0:<8s}'.format(str(obsInfo.majorPA)) +'\n'+
                      '{0:<10s}'.format('DELTAPA') + '{0:<8s}'.format('15') +'\n'+
                      '{0:<10s}'.format('Z0') + '{0:<8s}'.format('.5') +'\n'+
                      '{0:<10s}'.format('FREE') + '{0:<8s}'.format('VROT VDISP INC PA') +'\n'+
                      '{0:<10s}'.format('NORM') + '{0:<8s}'.format('LOCAL') +'\n'+
                      '{0:<10s}'.format('MASK') + '{0:<8s}'.format('NONE') +'\n'+
                      '{0:<10s}'.format('LTYPE') + '{0:<8s}'.format('2') +'\n'+
                      '{0:<10s}'.format('FTYPE') + '{0:<8s}'.format('2') +'\n'+
                      '{0:<10s}'.format('DISTANCE') + '{0:<8s}'.format(str(obsInfo.distance)) +'\n'+
                      '{0:<10s}'.format('BWEIGHT') + '{0:<8s}'.format('1') +'\n'+
                      '{0:<10s}'.format('WFUNC') + '{0:<8s}'.format('2') +'\n'+
                      '{0:<10s}'.format('LINEAR') + '{0:<8s}'.format('1.699') +'\n'+  
                      '{0:<10s}'.format('TWOSTAGE') + '{0:<8s}'.format('TRUE') +'\n'+  
                      '\n')
        outFile.close()

    print obsInfo.objectName, lineName
    break

circinus oiii88


In [5]:
from astropy.io import fits
h = fits.open(fitsFile)

In [6]:
h.info()

Filename: /Volumes/QbertPrimary/umdResearch/adapProposalNearby/pySpecKitCube/run4/circinus/1arc/outFitsFiles/1342191297_circinus_oiii88.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     365   ()      
  1  image         1 ImageHDU        70   (70, 71, 573)   float64   
  2  error         1 ImageHDU        70   (70, 71, 573)   float64   
  3  wcs-tab       1 BinTableHDU     37   1R x 2C   [573D, 573J]   
  4  flag          1 ImageHDU        58   (70, 71, 573)   int16   
  5  History       1 ImageHDU        23   ()      
  6  HistoryScript    1 BinTableHDU     44   75R x 1C   [185A]   
  7  HistoryTasks    1 BinTableHDU     46   43R x 4C   [1K, 31A, 1K, 9A]   
  8  HistoryParameters    1 BinTableHDU     74   431R x 10C   [1K, 27A, 13A, 20A, 1L, 1K, 1L, 58A, 11A, 42A]   
  9  flux          1 ImageHDU         8   (70, 71)   float64   
 10  wAsym         1 ImageHDU         8   (70, 71)   float64   
 11  w1            1 ImageHDU         8 

In [8]:
h[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                   71                                                  
NAXIS2  =                   71                                                  
NAXIS3  =                  321                                                  
LONGSTRN= 'OGIP 1.0'           / The OGIP long string convention may be used.   
COMMENT This FITS file may contain long string keyword values that are          
COMMENT continued over multiple keywords.  This convention uses the  '&'        
COMMENT character at the end of a string which is then continued                
COMMENT on subsequent keywords whose name = 'CONTINUE'.                         
         -------------- Herschel Structure Data--------------------             
         Following fields ar